In [84]:
import pandas as pd
import numpy as np

In [85]:
atlas = """

|             | ATLAS  | Recasting  |   ATLAS | Recasting  |
| ----------- | ------ | ---------- | ------- | ---------- |
|  **mGluino, mLSP, tau**             | **2000,100,1ns**  | **2000,100,1ns**  |   **2000,1800,1ns** | **2000,1800,1ns**  |
| Initial Events                 |   32   |   32   |   32   |   32  |
| Trigger-based data reduction   |   32   |        |   27   |       |
| Event cleaning                 |   32   |        |   27   |       |
| Good Runs List   |   32   |   --   |   27   |   --  |
| Primary vertex   |   32   |   --   |   27   |   --  |
| NCB veto   |   32   |   --   |   26   |   --  |
| MET trigger   |   31   |   --   |   24   |   --  |
| MET filter (MET > 200 GeV for recast)   |   31   |   30.4   |   17   |   15  |
| Offline MET (event selection eff. for recast)   |   29   |   26.8   |   7   |  9.3  |
|  |
| DV fiducial acceptance   |   28   |   --   |   6   |   --  |
| DV fit quality   |   27   |   --   |   6   |   --  |
| DV displacement   |   27   |   --   |   6   |   --  |
| Material veto   |   22   |   --   |   5   |   --  |
| Disabled module veto   |   22   |   --   |   5   |   --  |
| DV track multplicity   |   15   |   --   |   3   |   --  |
| DV mass (vertex level eff. for recast)   |   14   |   14.4   |   2   |   3.6  | """

keys = []
vals = {'mLLP' : [2000.0,2000.0], 'mLSP' : [100.,1800.], 'tau_ns' : [1.0,1.0]}
for l in atlas.splitlines():
    if '|' not in l: continue
    if '---' in l: continue
    if not l.strip(): continue
    key = l.split('|')[1]
    if not key.strip(): continue
    keys.append(key.strip())
vals.update({k : [] for k in keys[1:]})

for l in atlas.splitlines():
    if '|' not in l: continue
    if '---' in l: continue
    if not l.strip(): continue
    key = l.split('|')[1]
    key = key.strip()
    if 'mGluino' in key: continue
    if not key.strip(): continue
    x1 = eval(l.split('|')[2])
    x2 = eval(l.split('|')[4])
    vals[key] = [x1,x2]

atlasDF = pd.DataFrame.from_dict(vals)
atlasDF.T

,0,1
mLLP,2000.0,2000.0
mLSP,100.0,1800.0
tau_ns,1.0,1.0
Initial Events,32.0,32.0
Trigger-based data reduction,32.0,27.0
Event cleaning,32.0,27.0
Good Runs List,32.0,27.0
Primary vertex,32.0,27.0
NCB veto,32.0,26.0
MET trigger,31.0,24.0


In [86]:
recastData = pd.read_pickle('./gluino_cutflow.pcl')
recastData = recastData[(recastData['mLLP'] == 2000.) & ((recastData['mLSP'] == 100.) | (recastData['mLSP'] == 1800.)) & (np.round(recastData['tau_ns']) == 1.0)]
kfactor  = atlasDF.iloc[0]['Initial Events']/recastData.iloc[0]['Total'][0]
for c in recastData.columns:
    if c in ['Luminosity (1/fb)','mLLP','mLSP','tau_ns','Total MC Events','VertexEff Strategy','width', 'Total xsec (pb)']:
        continue
    recastData[c] = kfactor*np.array(recastData[c].to_list())[:,0]

renameCols = {'$MET > 200$ GeV' : 'MET filter (MET > 200 GeV for recast)', 'Jet Selection' : 'Offline MET (event selection eff. for recast)', '+DV Eff' : 'DV mass (vertex level eff. for recast)', 'Total' : 'Initial Events'}


recastData.rename(columns=renameCols,inplace=True)
recastData.reset_index(drop=True, inplace=True)
recastData.T

,0,1
mLLP,2.000000e+03,2.000000e+03
mLSP,1.000000e+02,1.800000e+03
width,6.582000e-16,6.582000e-16
tau_ns,1.000000e+00,1.000000e+00
Total MC Events,3.500200e+04,3.480400e+04
Total xsec (pb),5.365048e-04,5.334699e-04
Initial Events,3.200000e+01,3.200000e+01
MET filter (MET > 200 GeV for recast),3.039040e+01,1.595840e+01
Offline MET (event selection eff. for recast),3.027200e+01,1.568960e+01
"$R_{xy},z <$ 300 mm",2.939200e+01,1.520960e+01


In [95]:
atlasDF['label'] = 'ATLAS'
recastData['label'] = 'Recast'
df = atlasDF.merge(recastData,how='outer')
df.drop(columns=[c for c in df.columns if c not in atlasDF.columns],inplace=True)
df.T


/tmp/ipykernel_679614/980498824.py:3: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  df = atlasDF.merge(recastData,how='outer')


,0,1,2,3
mLLP,2000.0,2000.0,2000.0,2000.0
mLSP,100.0,1800.0,100.0,1800.0
tau_ns,1.0,1.0,1.0,1.0
Initial Events,32.0,32.0,32.0,32.0
Trigger-based data reduction,32.0,27.0,NaN,NaN
Event cleaning,32.0,27.0,NaN,NaN
Good Runs List,32.0,27.0,NaN,NaN
Primary vertex,32.0,27.0,NaN,NaN
NCB veto,32.0,26.0,NaN,NaN
MET trigger,31.0,24.0,NaN,NaN


In [96]:
for index in atlasDF.index.values:
    print(index)

0
1


In [123]:
combData = atlasDF.T.copy()
for index in atlasDF.T.index.values:
    atlasVals = atlasDF.T.loc[index]
    if index == 'label':
        combData.loc[index] = ['Recast (ATLAS)']*len(atlasVals)
        continue
    if index in ['mLLP','mLSP','tau_ns']:
        continue
    if isinstance(atlasVals[0],str):
        continue
    if index in recastData.columns:
        recastVals = recastData.T.loc[index]
    else:
        recastVals = [np.nan]*len(atlasVals)
    merged = ['%1.2f (%1.2f)' %x for x in zip(recastVals,atlasVals)]
    merged = [x.replace('nan', '-----') for  x in merged[:]]
    # print(index)
    # print(merged)
    combData.loc[index] = merged

In [124]:
print(combData.to_markdown())

|                                               | 0              | 1              |
|:----------------------------------------------|:---------------|:---------------|
| mLLP                                          | 2000.0         | 2000.0         |
| mLSP                                          | 100.0          | 1800.0         |
| tau_ns                                        | 1.0            | 1.0            |
| Initial Events                                | 32.00 (32.00)  | 32.00 (32.00)  |
| Trigger-based data reduction                  | ----- (32.00)  | ----- (27.00)  |
| Event cleaning                                | ----- (32.00)  | ----- (27.00)  |
| Good Runs List                                | ----- (32.00)  | ----- (27.00)  |
| Primary vertex                                | ----- (32.00)  | ----- (27.00)  |
| NCB veto                                      | ----- (32.00)  | ----- (26.00)  |
| MET trigger                                   | ----- (31.00)  | ----- (24